# Lab: Model Context Protocol (MCP)

## Introduction

The Model Context Protocol (MCP) provides a standardized approach for communication between AI-powered applications, particularly Large Language Models (LLMs). As LLM models become more sophisticated, capable of understanding complex **prompts**, utilizing external **tools**, and accessing diverse **resources**, MCP aims to simplify these interactions by offering a consistent interface.

This lab explores MCP's fundamental concepts, including its architecture and core components for managing the flow of **prompts**, data, and instructions for tool use. You'll learn how MCP functions over different transport mechanisms to facilitate structured and capable interactions with AI models.

### Model Context Protocol (MCP)

MCP is designed to define a clear structure for managing LLM **prompts**, contextual information (which can include references to **tools** and **resources**), and data exchange when an application (a client) communicates with a model service (a server). This is not the complete specification of the protocol, but a simplified overview to illustrate the key concepts.

Key objectives and potential benefits of using a protocol like MCP include:

* **Improved Interoperability**: Facilitating connections between applications and a variety of model backends.
* **Standardized Patterns**: Promoting common communication methods for leveraging **prompts**, requesting **tool** executions, and receiving model-generated content.
* **Rich Context & Instruction Handling**: Providing defined ways to pass not just conversational history, but also detailed **prompts**, and specifications for how models should use available **tools** or access **resources**.
* **Extensibility**: Allowing for future enhancements or specialized data exchanges within the protocol's framework.

MCP adapts established communication principles to the specific needs of AI model interactions, such as managing conversational context, interpreting detailed **prompts**, orchestrating **tool** use, and optionally streaming data.

#### Architecture

MCP interactions use a client-server architecture:

<div align="left">
  <img src="pictures/mcp-arch.png" alt="MCP Architecture" width="700">
</div>

1.  **Client**: An application that consumes model services. It forms and sends requests (containing **prompts**, data, and potentially **tool** calls) to the MCP server and processes the received responses.
2.  **Server**: A service that exposes one or more AI models via MCP. It listens for client requests, interprets **prompts**, manages **tool** execution or **resource** access as defined by the protocol, interacts with the model(s), and returns responses according to the protocol.
3.  **Transport Layer**: The underlying mechanism for message transmission. MCP defines the structure and interaction flow, with flexibility in transport mechanisms. MCP supports:
    * Standard Input/Output (STDIO)
    * Streamable HTTP (with optional Server-Sent Events)

This separation allows flexibility in choosing a transport suitable for the application's environment.

#### Components

MCP interactions involve several key elements and types of information:

1.  **Messages**: Defined units of information exchanged between client and server using JSON-RPC format. Common patterns include:
    * Request messages from client to server
    * Response messages from server to client  
    * Notification messages that don't require responses
    * Error messages for communicating issues

2.  **Prompts**: The core input that guides the model's behavior. MCP facilitates the clear transmission of prompts from the client to the server.

3.  **Tools & Resources**: Mechanisms by which a model can interact with external systems or data:
    * **Tools**: Pre-defined functions or capabilities that the model can be instructed to use (e.g., calculators, search engines, database query functions)
    * **Resources**: External data sources or knowledge bases that the model might need to access to fulfill a request
    * *Note: The specifics of how tools and resources are defined and invoked can vary between MCP implementations*

#### Protocol (Interaction Flow)

A typical interaction sequence in MCP includes:

1.  **Connection**: The client connects to the MCP server via the chosen transport mechanism.
2.  **Initialization**: Client and server exchange initialization messages to establish capabilities and protocol version.
3.  **Request**: The client sends a request message with the **prompt**, relevant context, parameters, and any **tool** or **resource** specifications.
4.  **Processing**: The server processes the request, which may involve:
    * Interpreting the prompt
    * Invoking tools as specified
    * Accessing resources as needed
    * Generating model responses
5.  **Response**: The server returns the response, either as a complete message or (if using Streamable HTTP) potentially as a stream of partial responses.
6.  **Error Handling**: If issues arise, appropriate error messages are exchanged.
7.  **Disconnection**: The connection may be closed or kept alive for subsequent interactions, depending on the transport and use case.

This structured approach helps create more predictable and maintainable integrations with AI models, encompassing complex interactions involving prompts, tools, and resources while maintaining flexibility in implementation details.

## Objectives

By the end of this lab, you will:

- Understand the core principles of the Model Context Protocol (MCP) and why it's valuable for LLM applications
- Explore different transport mechanisms for MCP implementation (STDIO, SSE, and HTTP)
- Implement basic MCP clients and servers using Python
- Learn how to integrate LLMs with MCP
- Gain practical experience with standardized protocols for AI communication

This lab provides hands-on experience with an emerging standard in the AI ecosystem, giving you the skills to build more interoperable and flexible LLM-powered applications.

## Getting Started

The following bullets must be ran prior to executing notebooks for running this lab:
  1. uv installed and available on PATH
      - Linux/MacOS:
          - `curl -sSL https://get.uv.dev | sh`
          - `source ~/.bashrc`
          - `curl -LsSf https://astral.sh/uv/install.sh | sh`
          - `source $HOME/.local/bin/env`

      - Windows:
          - `powershell -ExecutionPolicy ByPass -c "irm https://astral.sh/uv/install.ps1 | iex"`
          - Completley close and re-open VSCode, essentailly just restart terminal.
              - In my experience doing a terminal restart without closing VSCode does not work.

  2. Python 3.12 installed and venv created
      - `uv python install 3.12`
      - `uv venv -p 3.12`
      - Activate the venv:
          - Linux/MacOS:
              - `source .venv/bin/activate`
          - Windows:
              - `.\.venv\Scripts\activate`
      - `uv pip install ipykernel`

  3. (Optional) In order to run the final labe section you need npx installed which requires Node.js (and npm if on linux)
      - Windows:
          - Simply use installer and include extra tools checkbox when installing Node.js https://nodejs.org/en/download.
      - Linux/MacOS:
          - Google how to install Node.js on your OS as there are many different ways to do it.

  4. Select the venv as the notebook kernel
  <div align="left">
    <img src="pictures/kernel.png" alt="VSCode Juypter UI hint" width="800">
  </div>

  

**MUST restart Juypter kernel if automated install dependencies cell is ran**
<div align="left">
  <img src="pictures/restart.png" alt="VSCode Juypter UI hint" width="800">
</div>

In [ ]:
"""Install platform chat & mcp dependencies."""
!cd ../../.. && uv pip install --quiet -e .[chat] mcp
# Shut down the kernel so user must restart it to apply new pip installations.
# This is a workaround for the fact that Jupyter does not automatically
# pick up new installations in the current kernel.
!echo "Kernel will shut down to apply new pip installations, manual restart required."
import os
os._exit(00)

In [ ]:
"""Setup background process manager for sse/http server demos."""
from src.notebooks.platform_services.lablib.bg_util import clear_port, start_background_process, kill_background_process, kill_all_background_processes
from time import sleep
import subprocess as sp
print("lablib.util process management functions are ready.")

In [ ]:
"""Set up the lab chat model."""
from src.notebooks.platform_services.lablib.env_util import set_services_env

_, _, _ = set_services_env()

print("Chat env initialized successfully.")

## STDIO Transport

The STDIO (Standard Input/Output) transport is the primary MCP implementation that clients should support. It uses standard input and output streams to exchange MCP messages between client and server processes.

### Key Features

- **Primary Transport**: Clients SHOULD support STDIO whenever possible
- **Subprocess Communication**: The client launches the MCP server as a subprocess
- **Newline-Delimited Messages**: JSON-RPC messages are separated by newlines

### How It Works

1. **Process Launch**: The client launches the MCP server as a subprocess
2. **Message Exchange**: 
   - The server reads JSON-RPC messages from its standard input (stdin)
   - The server sends messages to its standard output (stdout)
   - Messages are delimited by newlines and MUST NOT contain embedded newlines
3. **Message Types**: Messages may be JSON-RPC requests, notifications, responses, or batched messages containing multiple requests/notifications
4. **Logging**: The server MAY write UTF-8 strings to its standard error (stderr) for logging purposes, which clients MAY capture, forward, or ignore
5. **Protocol Compliance**: 
   - The server MUST NOT write anything to stdout that is not a valid MCP message
   - The client MUST NOT write anything to the server's stdin that is not a valid MCP message

In [ ]:
"""Run the STDIO demo"""
stdio_client_command = "uv run lablib/mcp/stdio/client.py"
client_result = sp.run(stdio_client_command, shell=True, capture_output=True, text=True)
print(client_result.stdout)
if client_result.stderr:
    print("--- Server Logs ---")
    print(client_result.stderr)


## SSE Transport (Deprecated)

Server-Sent Events (SSE) was originally used in MCP as part of the "HTTP+SSE transport" protocol, which has since been deprecated and replaced by the Streamable HTTP transport.

### Important Note

- The standalone "HTTP+SSE transport" from protocol version 2024-11-05 has been **deprecated**
- SSE functionality has been **absorbed into** the new Streamable HTTP transport (as of protocol version 2025-03-26)
- The Python SDK currently only supports SSE transport
- This section covers the SSE implementation as used in current Python MCP implementations

### Key Features

- **Server-to-Client Streaming**: Enables servers to push updates to clients over HTTP
- **Event-Based Communication**: Uses SSE event format with `event`, `data`, and `id` fields
- **HTTP-Compatible**: Works over standard HTTP connections

### How It Works

1. Client establishes an HTTP connection requesting SSE stream
2. Server sends initial `endpoint` event with connection details
3. Server streams JSON-RPC messages as SSE events
4. Client processes messages as they arrive

In [ ]:
"""Run the SSE demo"""
# Clear port 8000 before starting the server
clear_port(8000)

sse_server_command = "uv run lablib/mcp/sse/server.py"
sse_client_command = "uv run lablib/mcp/sse/client.py"

print(f"Starting SSE server: {sse_server_command}")
server_proc = start_background_process("sse_server", sse_server_command)

if server_proc and server_proc.poll() is None:
    print(f"Running SSE client: {sse_client_command}")
    client_result = sp.run(sse_client_command, shell=True, capture_output=True, text=True)
    print("--- SSE Client Output ---")
    print(client_result.stdout)
    if client_result.stderr:
        print("--- SSE Client Errors ---")
        print(client_result.stderr)
    print("-------------------------")
else:
    print("SSE server failed to start or exited prematurely. Client will not run.")

print("Killing SSE server...")
kill_background_process("sse_server")

In [ ]:
"""Run the SSE Copilot demo"""
sse_server_command = "uv run --active lablib/mcp/sse/server.py"

print(f"Starting SSE server: {sse_server_command}")
server_proc = start_background_process("sse_server", sse_server_command)

output = """
        "my-test-mcp": {
            "url": "http://localhost:8000/math/sse"
        }
"""

print(f"Add the following to your llmesh/.vscode/mcp.json file:\n{output}")

In [ ]:
"""Cleanup sse process"""
print("Manually killing sse server...")
kill_background_process("sse_server")

## Streamable HTTP Transport

The Streamable HTTP transport is the current standard MCP transport mechanism, replacing the deprecated HTTP+SSE transport.

### Key Features

- **Current Standard**: The recommended transport mechanism for MCP
- **HTTP-Based**: Uses standard HTTP POST and GET requests
- **Optional Streaming**: Server can optionally use Server-Sent Events (SSE) for streaming responses
- **Stateless or Stateful**: Supports both basic stateless servers and more feature-rich servers with sessions
- **Multiple Connections**: Clients can maintain multiple concurrent connections

### How It Works

1. **Single Endpoint**: Server provides one HTTP endpoint that supports both POST and GET methods
2. **Sending Messages**: 
   - Client sends JSON-RPC messages via HTTP POST to the MCP endpoint
   - Client must include `Accept` header with both `application/json` and `text/event-stream`
3. **Server Response Options**:
   - **Simple Response**: Return `Content-Type: application/json` with a single JSON object
   - **Streaming Response**: Return `Content-Type: text/event-stream` to initiate SSE streaming
4. **Receiving Messages**: 
   - Client can issue HTTP GET to open SSE streams for server-initiated messages
   - Server may send JSON-RPC requests and notifications via SSE
5. **Session Management**: Server may optionally assign session IDs for stateful interactions

**Note**: The MCP Inspector (shown below) provides a TypeScript client for testing Streamable HTTP connections.

<div align="left">
  <img src="pictures/inspector.png" alt="Inspector screen shot" width="800">
</div>

In [ ]:
"""Run the HTTP demo"""

# Ensure background processes are killed before starting new ones
kill_all_background_processes()

# Ensure ports are clear before starting the server
clear_port(8000)
clear_port(6274)

# Start the HTTP server
start_background_process("mcp_server", "uv run lablib/mcp/streamable/server.py")

# Start the mcp inspector
start_background_process("mcp_inspector", "npx @modelcontextprotocol/inspector")

print("Starting MCP inspector and server...")
sleep(5)
print("MCP inspector UI available at http://localhost:6274/")
print("MCP Streamable HTTP demo server available at http://localhost:8000/math/mcp")


In [ ]:
# Manually kill streamable-http server and MCP inspector
# Will be automatically killed on kernel restart/exit
print("Manually terminating MCP servers...")
kill_background_process("mcp_server")
kill_background_process("mcp_inspector")

In [ ]:
# To kill ALL processes managed by lablib.util (if you started others):
print("Manually terminating ALL lablib-managed background processes...")
kill_all_background_processes()